# Data handling and pre-processing

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
#import math
from sklearn.model_selection import train_test_split

# Load data
file_paths = [
    '/content/drive/MyDrive/Datasets HAR/pamap2+physical+activity+monitoring/PAMAP2_Dataset/Protocol/subject101.dat',
    '/content/drive/MyDrive/Datasets HAR/pamap2+physical+activity+monitoring/PAMAP2_Dataset/Protocol/subject102.dat',
    '/content/drive/MyDrive/Datasets HAR/pamap2+physical+activity+monitoring/PAMAP2_Dataset/Protocol/subject103.dat',
    '/content/drive/MyDrive/Datasets HAR/pamap2+physical+activity+monitoring/PAMAP2_Dataset/Protocol/subject104.dat',
    '/content/drive/MyDrive/Datasets HAR/pamap2+physical+activity+monitoring/PAMAP2_Dataset/Protocol/subject105.dat',
    '/content/drive/MyDrive/Datasets HAR/pamap2+physical+activity+monitoring/PAMAP2_Dataset/Protocol/subject106.dat',
    '/content/drive/MyDrive/Datasets HAR/pamap2+physical+activity+monitoring/PAMAP2_Dataset/Protocol/subject107.dat',
    '/content/drive/MyDrive/Datasets HAR/pamap2+physical+activity+monitoring/PAMAP2_Dataset/Protocol/subject108.dat',
    '/content/drive/MyDrive/Datasets HAR/pamap2+physical+activity+monitoring/PAMAP2_Dataset/Protocol/subject109.dat'
    # Add paths to other subject files as needed
]

subjectID = [1,2,3,4,5,6,7,8,9]

activityIDdict = {0: 'transient',
              1: 'lying',
              2: 'sitting',
              3: 'standing',
              4: 'walking',
              5: 'running',
              6: 'cycling',
              7: 'Nordic_walking',
              9: 'watching_TV',
              10: 'computer_work',
              11: 'car driving',
              12: 'ascending_stairs',
              13: 'descending_stairs',
              16: 'vacuum_cleaning',
              17: 'ironing',
              18: 'folding_laundry',
              19: 'house_cleaning',
              20: 'playing_soccer',
              24: 'rope_jumping' }

colNames = ["timestamp", "activityID","heartrate"]

IMUhand = ['handTemperature',
           'handAcc16_1', 'handAcc16_2', 'handAcc16_3',
           'handAcc6_1', 'handAcc6_2', 'handAcc6_3',
           'handGyro1', 'handGyro2', 'handGyro3',
           'handMagne1', 'handMagne2', 'handMagne3',
           'handOrientation1', 'handOrientation2', 'handOrientation3', 'handOrientation4']

IMUchest = ['chestTemperature',
           'chestAcc16_1', 'chestAcc16_2', 'chestAcc16_3',
           'chestAcc6_1', 'chestAcc6_2', 'chestAcc6_3',
           'chestGyro1', 'chestGyro2', 'chestGyro3',
           'chestMagne1', 'chestMagne2', 'chestMagne3',
           'chestOrientation1', 'chestOrientation2', 'chestOrientation3', 'chestOrientation4']

IMUankle = ['ankleTemperature',
           'ankleAcc16_1', 'ankleAcc16_2', 'ankleAcc16_3',
           'ankleAcc6_1', 'ankleAcc6_2', 'ankleAcc6_3',
           'ankleGyro1', 'ankleGyro2', 'ankleGyro3',
           'ankleMagne1', 'ankleMagne2', 'ankleMagne3',
           'ankleOrientation1', 'ankleOrientation2', 'ankleOrientation3', 'ankleOrientation4']

columns = colNames + IMUhand + IMUchest + IMUankle  #all columns in one list

#len(columns)


dataCollection = pd.DataFrame()

for file in file_paths:
    procData = pd.read_table(file, header=None, sep='\s+')
    procData.columns = columns
    procData['subject_id'] = int(file[-5])
    dataCollection = pd.concat([dataCollection, procData], ignore_index=True)

dataCollection.reset_index(drop=True, inplace=True)
#dataCollection.head()

'''
unique_classes = dataCollection['activityID'].unique()
print(f"Number of unique classes: {len(unique_classes)}")
print(f"Unique classes: {unique_classes}")
'''

def dataCleaning(dataCollection):
        dataCollection = dataCollection.drop(['handOrientation1', 'handOrientation2', 'handOrientation3', 'handOrientation4',
                                             'chestOrientation1', 'chestOrientation2', 'chestOrientation3', 'chestOrientation4',
                                             'ankleOrientation1', 'ankleOrientation2', 'ankleOrientation3', 'ankleOrientation4'],
                                             axis = 1)  # removal of orientation columns as they are not needed
        dataCollection = dataCollection.drop(dataCollection[dataCollection.activityID == 0].index) #removal of any row of activity 0 as it is transient activity which it is not used
        dataCollection = dataCollection.apply(pd.to_numeric, errors = 'coerce') #removal of non numeric data in cells
        dataCollection = dataCollection.interpolate() #removal of any remaining NaN value cells by constructing new data points in known set of data points

        return dataCollection


dataCol = dataCleaning(dataCollection)
dataCol.reset_index(drop = True, inplace = True)
#dataCol.head(10)

dataCol.isnull().sum()
for i in range(0,4):
    dataCol["heartrate"].iloc[i]=100



train_df = dataCol.sample(frac=0.8, random_state=1)
test_df = dataCol.drop(train_df.index)

#train_df.describe()
train_df = train_df.drop(["timestamp", "subject_id"],axis=1)

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler,RobustScaler

#scaling all columns except subject and activity
scaler = RobustScaler()
df_scaled = train_df.copy()
df_scaled_test = test_df.copy()

df_scaled.iloc[:,1:41] = scaler.fit_transform(df_scaled.iloc[:,1:41])
df_scaled_test.iloc[:,1:41] = scaler.fit_transform(df_scaled_test.iloc[:,1:41])

#df_scaled.head()

X_train = df_scaled.drop('activityID', axis=1).values
y_train = df_scaled['activityID'].values

# Test Dataset
X_test = df_scaled.drop('activityID', axis=1).values
y_test = df_scaled['activityID'].values

from sklearn.decomposition import PCA
pca = PCA()
pca.fit(X_train)
var= pca.explained_variance_ratio_
var1=np.cumsum(np.round(pca.explained_variance_ratio_, decimals=4)*100)
'''
plt.title("PCA Variance against num of Componmnets")
plt.ylabel("Variance %")
plt.xlabel("Number of components")
l = plt.axhline(94, color="red")

plt.plot(var1)
plt.grid()
'''

pca = PCA(n_components=17)
X_train=pca.fit_transform(X_train)
X_test=pca.fit_transform(X_test)


# Reshaped to match input shape
X_train_reshaped = np.expand_dims(X_train, axis=-1)
X_test_reshaped = np.expand_dims(X_test, axis=-1)

print(X_train_reshaped.shape)
print(X_test_reshaped.shape)

import tensorflow as tf

unique_classes = [0, 1, 2, 3, 17, 16, 12, 13, 4, 7, 6, 5, 24]
class_mapping = {old: new for new, old in enumerate(unique_classes)} #sequentially mapped for one_hot_encoding
y_train_mapped = [class_mapping[label] for label in y_train]
y_test_mapped = [class_mapping[label] for label in y_test]

# One-hot encoding
num_classes = 13
y_train_encoded = tf.keras.utils.to_categorical(y_train_mapped, num_classes)
y_test_encoded = tf.keras.utils.to_categorical(y_test_mapped, num_classes)

use_subset = True

if use_subset:
    X_train_subset, _, y_train_subset, _ = train_test_split(X_train_reshaped, y_train_encoded, test_size=0.9, random_state=42)
    X_test_subset, _, y_test_subset, _ = train_test_split(X_test_reshaped, y_test_encoded, test_size=0.9, random_state=42)
    using_train_x = X_train_subset
    using_train_y = y_train_subset
    using_test_x = X_test_subset
    using_test_y = y_test_subset
else:
    using_train_x = X_train_reshaped
    using_train_y = y_train_encoded
    using_test_x = X_test_reshaped
    using_test_y = y_test_encoded


<ipython-input-1-f465a8d342e0>:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataCol["heartrate"].iloc[i]=100
<ipython-input-1-f465a8d342e0>:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataCol["heartrate"].iloc[i]=100
<ipython-input-1-f465a8d342e0>:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataCol["heartrate"].iloc[i]=100
<ipython-input-1-f465a8d342e0>:107: SettingWithCopyWarning: 
A value is trying to be set on a 

(1554298, 17, 1)
(1554298, 17, 1)


---

# CNN-LSTM

In [ ]:
import time
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, LSTM, Dense, concatenate, MaxPooling1D, Dropout, BatchNormalization, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

def create_c_lstm(input_shape,num_classes):
  inputs = Input(shape=input_shape)
  conv1 = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(0.01))(inputs)
  conv2 = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(0.01))(inputs)
  conv3 = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(0.01))(inputs)
  concat_conv = concatenate([conv1, conv2, conv3], axis=-1)
  reshape = Reshape((concat_conv.shape[1], -1))(concat_conv)
  lstm = LSTM(128)(reshape)
  dense = Dense(64, activation='tanh')(lstm)
  outputs = Dense(num_classes, activation='softmax')(dense)

  model = Model(inputs, outputs)
  return model


# Model creation
c_lstm_model = create_c_lstm(using_train_x.shape[1:], num_classes)
c_lstm_model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['mae'])
c_lstm_model.summary()

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
#model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

# Training
history = c_lstm_model.fit(using_train_x, using_train_y, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping,reduce_lr])

# Evaluation
y_pred = c_lstm_model.predict(using_test_x)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(using_test_y, axis=1)

accuracy = accuracy_score(y_true, y_pred_classes)
precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

# Inference time
start_time = time.time()
predictions = c_lstm_model.predict(using_test_x)
end_time = time.time()
inference_time = end_time - start_time
print(f'Inference Time(C-LSTM): {inference_time} seconds')



# **MULTISCALE CNN-LSTM MODELS**

IMC-LSTM  
**updated**

In [ ]:
import time
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, LSTM, Dense, concatenate, MaxPooling1D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


def create_imc_lstm(input_shape, num_classes):
    inputs = Input(shape=input_shape)

    # Independent convolutional layers
    conv1 = Conv1D(filters=64, kernel_size=3, activation='relu')(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = MaxPooling1D(pool_size=2)(conv1)

    conv2 = Conv1D(filters=64, kernel_size=5, activation='relu')(inputs)
    conv2 = BatchNormalization()(conv2)
    conv2 = MaxPooling1D(pool_size=2)(conv2)

    conv3 = Conv1D(filters=64, kernel_size=7, activation='relu')(inputs)
    conv3 = BatchNormalization()(conv3)
    conv3 = MaxPooling1D(pool_size=2)(conv3)

    # Independent LSTM layers
    lstm1 = LSTM(128, return_sequences=True)(conv1)
    lstm1 = LSTM(128)(lstm1)

    lstm2 = LSTM(128, return_sequences=True)(conv2)
    lstm2 = LSTM(128)(lstm2)

    lstm3 = LSTM(128, return_sequences=True)(conv3)
    lstm3 = LSTM(128)(lstm3)

    # Concatenate LSTM outputs
    concat = concatenate([lstm1, lstm2, lstm3])

    # Final dense layer
    dropout = Dropout(0.2)(concat)
    dense = Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(dropout)
    outputs = Dense(num_classes, activation='softmax')(dense)

    model = Model(inputs, outputs)
    return model

# Model creation
imc_lstm_model = create_imc_lstm(using_train_x.shape[1:], num_classes)
imc_lstm_model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['mae'])
imc_lstm_model.summary()

# Callbacks
early_stopping = EarlyStopping(monitor='accuracy', patience=10, restore_best_weights=True)
#model_checkpoint = ModelCheckpoint('bestIMC_model.h5', save_best_only=True, monitor='val_loss')
reduce_lr = ReduceLROnPlateau(monitor='accuracy', factor=0.5, patience=5, min_lr=1e-6)

# Training
history = imc_lstm_model.fit(using_train_x, using_train_y, epochs=1, batch_size=32, validation_split=0.2, callbacks=[early_stopping, reduce_lr]) #PASS MODEL_CHECKPOINT HERE IF THE MODEL IS TO BE SAVED


# Evaluation
y_pred = imc_lstm_model.predict(using_test_x)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(using_test_y, axis=1)

accuracy = accuracy_score(y_true, y_pred_classes)
precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

# Inference time
start_time = time.time()
predictions = imc_lstm_model.predict(using_test_x)
end_time = time.time()
inference_time = end_time - start_time
print(f'Inference Time(IMC-LSTM): {inference_time} seconds')


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 17, 1)]              0         []                            
                                                                                                  
 conv1d (Conv1D)             (None, 15, 64)               256       ['input_1[0][0]']             
                                                                                                  
 conv1d_1 (Conv1D)           (None, 13, 64)               384       ['input_1[0][0]']             
                                                                                                  
 conv1d_2 (Conv1D)           (None, 11, 64)               512       ['input_1[0][0]']             
                                                                                              

KeyboardInterrupt: 

In [ ]:
#sample = np.expand_dims(using_test_x[0], axis=0)

import matplotlib.pyplot as plt

# Assuming conv1, conv2, and conv3 are the layers we want to visualize

# Function to plot feature maps
def plot_feature_maps(model, layer_names, sample):
    layer_outputs = [model.get_layer(name).output for name in layer_names]
    intermediate_model = Model(inputs=model.input, outputs=layer_outputs)
    feature_maps = intermediate_model.predict(np.expand_dims(sample, axis=0))

    for fmap, name in zip(feature_maps, layer_names):
        num_filters = fmap.shape[-1]
        fig, axes = plt.subplots(1, num_filters, figsize=(20, 15))
        fig.suptitle(f'Feature maps from layer: {name}')
        for i in range(num_filters):
            axes[i].plot(fmap[0, :, i])
            axes[i].axis('off')
        plt.show()

# Plot feature maps
sample_index = 0  # Change this to the index of the sample you want to visualize
plot_feature_maps(imc_lstm_model, ['conv1d_1', 'conv1d_2', 'conv1d'], using_test_x[sample_index])


CMC-LSTM

**updated**

In [ ]:
import time
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, LSTM, Dense, concatenate, MaxPooling1D, Dropout, BatchNormalization, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

def create_cmc_lstm(input_shape, num_classes):
    inputs = Input(shape=(input_shape[0], 1))

    # independent convolutional layers
    conv1 = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(0.01))(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = MaxPooling1D(pool_size=2)(conv1)

    conv2 = Conv1D(filters=64, kernel_size=5, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(0.01))(inputs)
    conv2 = BatchNormalization()(conv2)
    conv2 = MaxPooling1D(pool_size=2)(conv2)

    conv3 = Conv1D(filters=64, kernel_size=7, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(0.01))(inputs)
    conv3 = BatchNormalization()(conv3)
    conv3 = MaxPooling1D(pool_size=2)(conv3)

    # Concatenate convolutional outputs
    concat_conv = concatenate([conv1, conv2, conv3], axis=-1)

    # Reshape to 3D before LSTM
    reshape = Reshape((concat_conv.shape[1], -1))(concat_conv)

    # Common/combined LSTM layer
    lstm = LSTM(128)(reshape)

    # Final dense layer
    dropout = Dropout(0.2)(lstm)
    dense = Dense(64, activation='tanh')(dropout)
    outputs = Dense(num_classes, activation='softmax')(dense)

    model = Model(inputs, outputs)
    return model

# Model creation
cmc_lstm_model = create_cmc_lstm(using_train_x.shape[1:], num_classes)
cmc_lstm_model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['mae'])
cmc_lstm_model.summary()

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
#model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

# Training
history = cmc_lstm_model.fit(using_train_x, using_train_y, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping,reduce_lr])

# Evaluation
y_pred = cmc_lstm_model.predict(using_test_x)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(using_test_y, axis=1)

accuracy = accuracy_score(y_true, y_pred_classes)
precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

# Inference time
start_time = time.time()
predictions = cmc_lstm_model.predict(using_test_x)
end_time = time.time()
inference_time = end_time - start_time
print(f'Inference Time(CMC-LSTM): {inference_time} seconds')


SMC-LSTM
**updated**

In [ ]:
import time
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, LSTM, Dense, concatenate, MaxPooling1D, Dropout, BatchNormalization, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# Custom layer for stacking convolutional outputs
class StackLayer(tf.keras.layers.Layer):
    def call(self, inputs):
        return tf.stack(inputs, axis=2)

def create_smc_lstm(input_shape, num_classes):
    inputs = Input(shape=(input_shape[0], 1))

    # Three convolutional layers
    conv1 = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = MaxPooling1D(pool_size=2)(conv1)

    conv2 = Conv1D(filters=64, kernel_size=5, activation='relu', padding='same')(inputs)
    conv2 = BatchNormalization()(conv2)
    conv2 = MaxPooling1D(pool_size=2)(conv2)

    conv3 = Conv1D(filters=64, kernel_size=7, activation='relu', padding='same')(inputs)
    conv3 = BatchNormalization()(conv3)
    conv3 = MaxPooling1D(pool_size=2)(conv3)

    # Stack the convolutional outputs along a new dimension using the custom layer
    stacked_conv_outputs = StackLayer()([conv1, conv2, conv3])

    # Combine CNN outputs into a single LSTM layer
    lstm_input_shape = (stacked_conv_outputs.shape[1] * stacked_conv_outputs.shape[2], stacked_conv_outputs.shape[3])
    reshaped_stacked_conv_outputs = Reshape(lstm_input_shape)(stacked_conv_outputs)
    lstm = LSTM(128)(reshaped_stacked_conv_outputs)

    # Final dense layers
    dropout = Dropout(0.2)(lstm)
    dense = Dense(128, activation='tanh')(dropout)
    outputs = Dense(num_classes, activation='softmax')(dense)

    model = Model(inputs, outputs)
    return model

# Model creation
smc_lstm_model = create_smc_lstm(using_train_x.shape[1:], num_classes)
smc_lstm_model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['mae'])
smc_lstm_model.summary()
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

# Training
history = smc_lstm_model.fit(using_train_x, using_train_y, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping, reduce_lr])

# Evaluation
y_pred = smc_lstm_model.predict(using_test_x)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(using_test_y, axis=1)

accuracy = accuracy_score(y_true, y_pred_classes)
precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
# Inference time
start_time = time.time()
predictions = smc_lstm_model.predict(using_test_x)
end_time = time.time()
inference_time = end_time - start_time
print(f'Inference Time(SMC-LSTM): {inference_time} seconds')


Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_8 (InputLayer)        [(None, 17, 1)]              0         []                            
                                                                                                  
 conv1d_21 (Conv1D)          (None, 17, 64)               256       ['input_8[0][0]']             
                                                                                                  
 conv1d_22 (Conv1D)          (None, 17, 64)               384       ['input_8[0][0]']             
                                                                                                  
 conv1d_23 (Conv1D)          (None, 17, 64)               512       ['input_8[0][0]']             
                                                                                            

KeyboardInterrupt: 



---

# **CNN-GRU**

In [ ]:
import time
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, GRU, Dense, concatenate, MaxPooling1D, Dropout, BatchNormalization, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

def create_c_gru(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    conv1 = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(0.01))(inputs)
    conv2 = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(0.01))(inputs)
    conv3 = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(0.01))(inputs)
    concat_conv = concatenate([conv1, conv2, conv3], axis=-1)
    reshape = Reshape((concat_conv.shape[1], -1))(concat_conv)
    gru = GRU(128)(reshape)
    dense = Dense(64, activation='tanh')(gru)
    outputs = Dense(num_classes, activation='softmax')(dense)

    model = Model(inputs, outputs)
    return model

# Model creation
c_gru_model = create_c_gru(using_train_x.shape[1:], num_classes)
c_gru_model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['mae'])
c_gru_model.summary()

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

# Training
history = c_gru_model.fit(using_train_x, using_train_y, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping, reduce_lr])

# Evaluation
y_pred = c_gru_model.predict(using_test_x)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(using_test_y, axis=1)

accuracy = accuracy_score(y_true, y_pred_classes)
precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

# Inference time
start_time = time.time()
predictions = c_gru_model.predict(using_test_x)
end_time = time.time()
inference_time = end_time - start_time
print(f'Inference Time(CMC-GRU): {inference_time} seconds')


# Multiscale CNN with GRUs





 **IMC-GRU**

In [ ]:
import time
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, GRU, Dense, concatenate, MaxPooling1D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

def create_imc_gru(input_shape, num_classes):
    inputs = Input(shape=input_shape)

    # Independent convolutional layers
    conv1 = Conv1D(filters=64, kernel_size=3, activation='relu')(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = MaxPooling1D(pool_size=2)(conv1)

    conv2 = Conv1D(filters=64, kernel_size=5, activation='relu')(inputs)
    conv2 = BatchNormalization()(conv2)
    conv2 = MaxPooling1D(pool_size=2)(conv2)

    conv3 = Conv1D(filters=64, kernel_size=7, activation='relu')(inputs)
    conv3 = BatchNormalization()(conv3)
    conv3 = MaxPooling1D(pool_size=2)(conv3)

    # Independent GRU layers
    gru1 = GRU(128, return_sequences=True)(conv1)
    gru1 = GRU(128)(gru1)

    gru2 = GRU(128, return_sequences=True)(conv2)
    gru2 = GRU(128)(gru2)

    gru3 = GRU(128, return_sequences=True)(conv3)
    gru3 = GRU(128)(gru3)

    # Concatenate GRU outputs
    concat = concatenate([gru1, gru2, gru3])

    # Final dense layer
    dropout = Dropout(0.2)(concat)
    dense = Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(dropout)
    outputs = Dense(num_classes, activation='softmax')(dense)

    model = Model(inputs, outputs)
    return model

# Model creation
imc_gru_model = create_imc_gru(using_train_x.shape[1:], num_classes)
imc_gru_model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['mae'])
imc_gru_model.summary()

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

# Training
history = imc_gru_model.fit(using_train_x, using_train_y, epochs=1, batch_size=32, validation_split=0.2, callbacks=[early_stopping, reduce_lr])

# Evaluation
y_pred = imc_gru_model.predict(using_test_x)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(using_test_y, axis=1)

accuracy = accuracy_score(y_true, y_pred_classes)
precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

# Inference time
start_time = time.time()
predictions = imc_gru_model.predict(using_test_x)
end_time = time.time()
inference_time = end_time - start_time
print(f'Inference Time(IMC-GRU): {inference_time} seconds')


 CMC-GRU

In [ ]:
import time
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, GRU, Dense, concatenate, MaxPooling1D, Dropout, BatchNormalization, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

def create_cmc_gru(input_shape, num_classes):
    inputs = Input(shape=(input_shape[0], 1))

    # independent convolutional layers
    conv1 = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(0.01))(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = MaxPooling1D(pool_size=2)(conv1)

    conv2 = Conv1D(filters=64, kernel_size=5, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(0.01))(inputs)
    conv2 = BatchNormalization()(conv2)
    conv2 = MaxPooling1D(pool_size=2)(conv2)

    conv3 = Conv1D(filters=64, kernel_size=7, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(0.01))(inputs)
    conv3 = BatchNormalization()(conv3)
    conv3 = MaxPooling1D(pool_size=2)(conv3)

    # Concatenate convolutional outputs
    concat_conv = concatenate([conv1, conv2, conv3], axis=-1)

    # Reshape to 3D before GRU
    reshape = Reshape((concat_conv.shape[1], -1))(concat_conv)

    # Common/combined GRU layer
    gru = GRU(128)(reshape)

    # Final dense layer
    dropout = Dropout(0.2)(gru)
    dense = Dense(64, activation='tanh')(dropout)
    outputs = Dense(num_classes, activation='softmax')(dense)

    model = Model(inputs, outputs)
    return model

# Model creation
cmc_gru_model = create_cmc_gru(using_train_x.shape[1:], num_classes)
cmc_gru_model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['mae'])
cmc_gru_model.summary()

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

# Training
history = cmc_gru_model.fit(using_train_x, using_train_y, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping, reduce_lr])

# Evaluation
y_pred = cmc_gru_model.predict(using_test_x)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(using_test_y, axis=1)

accuracy = accuracy_score(y_true, y_pred_classes)
precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

# Inference time
start_time = time.time()
predictions = cmc_gru_model.predict(using_test_x)
end_time = time.time()
inference_time = end_time - start_time
print(f'Inference Time(CMC-GRU): {inference_time} seconds')


SMC-GRU

In [ ]:
import time
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, GRU, Dense, concatenate, MaxPooling1D, Dropout, BatchNormalization, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Custom layer for stacking convolutional outputs
class StackLayer(tf.keras.layers.Layer):
    def call(self, inputs):
        return tf.stack(inputs, axis=2)

def create_smc_gru(input_shape, num_classes):
    inputs = Input(shape=(input_shape[0], 1))

    # Three convolutional layers
    conv1 = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = MaxPooling1D(pool_size=2)(conv1)

    conv2 = Conv1D(filters=64, kernel_size=5, activation='relu', padding='same')(inputs)
    conv2 = BatchNormalization()(conv2)
    conv2 = MaxPooling1D(pool_size=2)(conv2)

    conv3 = Conv1D(filters=64, kernel_size=7, activation='relu', padding='same')(inputs)
    conv3 = BatchNormalization()(conv3)
    conv3 = MaxPooling1D(pool_size=2)(conv3)

    # Stack the convolutional outputs along a new dimension using the custom layer
    stacked_conv_outputs = StackLayer()([conv1, conv2, conv3])

    # Combine CNN outputs into a single GRU layer
    lstm_input_shape = (stacked_conv_outputs.shape[1] * stacked_conv_outputs.shape[2], stacked_conv_outputs.shape[3])
    reshaped_stacked_conv_outputs = Reshape(lstm_input_shape)(stacked_conv_outputs)
    gru = GRU(128)(reshaped_stacked_conv_outputs)

    # Final dense layers
    dropout = Dropout(0.2)(gru)
    dense = Dense(128, activation='tanh')(dropout)
    outputs = Dense(num_classes, activation='softmax')(dense)

    model = Model(inputs, outputs)
    return model

# Model creation
smc_gru_model = create_smc_gru(using_train_x.shape[1:], num_classes)
smc_gru_model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['mae'])
smc_gru_model.summary()

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

# Training
history = smc_gru_model.fit(using_train_x, using_train_y, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping, reduce_lr])

# Evaluation
y_pred = smc_gru_model.predict(using_test_x)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(using_test_y, axis=1)

accuracy = accuracy_score(y_true, y_pred_classes)
precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

# Inference time
start_time = time.time()
predictions = smc_gru_model.predict(using_test_x)
end_time = time.time()
inference_time = end_time - start_time
print(f'Inference Time(SMC-GRU): {inference_time} seconds')


In [ ]:
def predict_function(model,input_data):
    return model(input_data)

# Warm-up run
predictions = predict_function(smc_lstm_model,using_test_x)

# Measure inference time
start_time = time.time()
predictions = predict_function(using_test_x)
end_time = time.time()
inference_time = end_time - start_time

print(f'Inference Time(SMC-GRU): {inference_time} seconds')


KeyboardInterrupt: 